## Install Libraries

Here we are 
- installing libraries, 
- downloading the crowdhuman dataset zip files, and 
- unzipping files to the correct directories.

In [ ]:
# Install mmdetection
!pip install -U openmim
!mim install mmengine
!mim install "mmcv>=2.0.0"
!mim install mmdet
!git clone https://github.com/open-mmlab/mmdetection.git

In [ ]:
# Download zips
!wget http://codingai-s3.s3.amazonaws.com/public/crowdhuman.zip
!wget http://codingai-s3.s3.amazonaws.com/public/crowdhuman_annotations.zip
!wget http://codingai-s3.s3.amazonaws.com/public/crowdhuman_demo_config.py

In [ ]:
# Unzip
!mkdir mmdetection/data
!unzip crowdhuman.zip -d mmdetection/data
!mv mmdetection/data/crowdhuman mmdetection/data/coco
!unzip crowdhuman_annotations.zip -d mmdetection/data/coco
!mv crowdhuman_demo_config.py mmdetection/.

## View Random Image in Labeled Dataset
Run the scripts below to view a random file from our dataset. 

In [ ]:
import os, json, cv2, glob, random
import matplotlib.pyplot as plt
def draw_annotations_on_image(file_name, annotations):
    # Read the image using OpenCV
    image = cv2.imread(file_name)
    
    # Loop through the annotations and draw the bounding boxes and labels
    for annotation in annotations:
        bbox = annotation[0]["bbox"]
        category = annotation[1]
        x, y, w, h = map(int, bbox)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 3)
        cv2.putText(image, category, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 3)
    
    # Save the image with the bounding boxes and labels to a new file
    #cv2.imwrite("new.jpg", image)
    return image
def get_annotations_for_image(file_name, annotations_dict):
    # Find the image ID corresponding to the given file name
    image_id = None
    for image in annotations_dict["images"]:
        if image["file_name"] == file_name:
            image_id = image["id"]
            break
    if image_id is None:
        raise ValueError(f"No image found with file name '{file_name}'")
    
    # Find all annotations related to the image
    annotations = []
    for annotation in annotations_dict["annotations"]:
        if annotation["image_id"] == image_id:
            category_id = annotation["category_id"]
            # Find the relevant category
            category = None
            for cat in annotations_dict["categories"]:
                if cat["id"] == category_id:
                    category = cat["name"]
                    break
            if category is None:
                raise ValueError(f"No category found with ID '{category_id}'")
            # Add the annotation and category as a tuple to the list
            annotations.append((annotation, category))
    
    return annotations
def get_random_file(mydir):
    files = glob.glob(os.path.join(mydir,'*'))
    i = random.randint(0,len(files)-1)
    return files[i]
def view_random_labeled_image(img_dir,annotation_file):
    filename = get_random_file(img_dir)
    subname = os.path.split(filename)[1]
    annotations_dict = json.load(open(annotation_file,'r'))
    annotations = get_annotations_for_image(subname,annotations_dict)

    image=draw_annotations_on_image(filename, annotations)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.clf()
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.axis('on')
    plt.show()

Each time you run it the cell, you should get a different image. Note: If you are getting some error, your directories may be places in the wrong place.

In [ ]:
ANNOTATION_FILE = 'mmdetection/data/coco/annotations/instances_train2017.json'
IMAGES_DIR = 'mmdetection/data/coco/train2017'

view_random_labeled_image(IMAGES_DIR,ANNOTATION_FILE)

## Train Your Object Detector
We have the dataset. We have a configuration. Let's train your custom object detector.

In [ ]:
cd /content/mmdetection

In [ ]:
!python tools/train.py crowdhuman_demo_config.py

## Running Inference with Your Model

In [ ]:
# Uncomment and use below if you did not train your own model

# !wget http://codingai-s3.s3.amazonaws.com/public/crowdhuman_epoch_1.pth
# !wget http://codingai-s3.s3.amazonaws.com/public/crowdhuman_demo_config.py
# !mv crowdhuman_epoch_1.pth /content/mmdetection/.
# !mv crowdhuman_demo_config.py /content/mmdetection/.

In [ ]:
import cv2
import mmcv
from mmcv.transforms import Compose
from mmengine.utils import track_iter_progress
from mmdet.registry import VISUALIZERS
from mmdet.apis import init_detector, inference_detector
import matplotlib.pyplot as plt

CONFIG= '/content/mmdetection/crowdhuman_demo_config.py'
CHECKPOINT = '/content/mmdetection/work_dirs/faster-rcnn_r50-caffe_fpn_ms-1x_coco-person/epoch_1.pth'
#CHECKPOINT = '/content/mmdetection/crowdhuman_epoch_1.pth' # Use this if you did not train your own model

# Build the model from a config file and a checkpoint file
model = init_detector(CONFIG, CHECKPOINT, device='cuda:0')
model.cfg.test_dataloader.dataset.pipeline[0].type = 'LoadImageFromNDArray'
test_pipeline = Compose(model.cfg.test_dataloader.dataset.pipeline)

# Visualizer is used to show results of detector
visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta = model.dataset_meta

In [ ]:
!wget http://codingai-s3.s3.amazonaws.com/public/sample2.png
IMAGE_PATH = 'sample2.png'

image = mmcv.imread(IMAGE_PATH)
image = mmcv.imconvert(image, 'bgr', 'rgb')
plt.clf()
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.axis('on')
plt.show()

In [ ]:
# Show the results

result = inference_detector(model, image, test_pipeline=test_pipeline)

visualizer.add_datasample(
        'result',
        image,
        data_sample=result,
        draw_gt=False)
vis_image = visualizer.get_image()
plt.figure(figsize=(10,10))
plt.imshow(vis_image)
plt.axis('on')
plt.show()

In [ ]:

instances = result.pred_instances.numpy().cpu()
print(instances)
print("")
print("Congrats! You've just run an image through your newly trained object detector.\n")
print("The details of the detections are found above\n")